# **데이터 분석 및 시각화 연습 예제-3**

# <font color="blue">1. 데이터프레임 조작하기 예제</font>

## <font color="green">2-1. 데이터 불러오기</font>

In [ ]:
import pandas as pd

In [ ]:
# colab에서 파일을 업로드하려면 아래의 코드를 사용
from google.colab import files
uploaded = files.upload()

In [ ]:
# 0번 열을 데이터프레임의 인덱스로 사용하고, 문자열 데이터를 날짜 데이터로 변환
air_quality = pd.read_csv("air_quality_no2_long.csv")
air_quality = air_quality.rename(columns={"date.utc": "datetime"})
air_quality.head()

## <font color="green">2-2. 시계열 데이터 다루기</font>

### <font color="red">2-2-1. 시계열 데이터 변환</font>

In [ ]:
air_quality["datetime"]

기본적으로 datetime 열의 데이터는 문자열이며 연도, 요일 추출 등의 날짜-시간 관련 연산을 제공하지 않는다.

---

`to_datetime` 함수를 적용함으로써, pandas는 문자열을 인식하고 날짜-시간(datetime64[ns, UTC]) 객체로 변환한다.

In [ ]:
air_quality["datetime"] = pd.to_datetime(air_quality["datetime"])

In [ ]:
air_quality["datetime"]

pandas에서는 날짜-시간 객체를 `pandas.Timestamp` 클래스 및 그 확장 클래스들로 처리하며, 이는 python 표준 라이브러리의 `datetime.datetime` 클래스와 유사하다.

In [ ]:
type(air_quality["datetime"].dtypes)

`read_csv` 및 `read_json` 함수에는 파일을 불러올 때 시계열 데이터로 변환할지 지정하는 옵션, *parse_dates*를 사용할 수 있다.

In [ ]:
# 파일을 불러오면서 date.utc 열을 지정해서 날짜 형식으로 변경
pd.read_csv("air_quality_no2_long.csv", parse_dates=["date.utc"]).info()

*pandas.Timestamp* 클래스 및 그 관련 클래스들을 사용하면 날짜-시간 연산이 가능하다.

In [ ]:
# 관측 기록 시작일 및 종료일 출력
air_quality["datetime"].min(), air_quality["datetime"].max()

In [ ]:
# 관측 기간 출력
air_quality["datetime"].max() - air_quality["datetime"].min()

`pandas.Timestamp` 클래스는 다양한 속성들을 제공하는데, *dt* 속성을 통해 특정 날짜-시간 관련 데이터를 추출할 수 있다.

In [ ]:
air_quality["datetime"].dt

In [ ]:
dir(air_quality["datetime"].dt)

In [ ]:
# 관측 날짜 데이터에서 연도 데이터만 추출해서 새로운 열 지정
air_quality["year"] = air_quality["datetime"].dt.year
air_quality

In [ ]:
# 관측 날짜 데이터에서 시각 데이터만 추출해서 새로운 열 지정
air_quality["hour"] = air_quality["datetime"].dt.hour
air_quality

### <font color="red">2-2-2. 시계열 데이터 그룹화</font>

`Dataframe.groupby` 함수를 사용할 때 그룹으로 묶을 기준 열을 제시해야 하는데, 이때 시계열 데이터에서 추출한 데이터를 지정할 수 있다.

In [ ]:
# 각 요일별, 관측소별 평균 측정값 구하기
air_quality.groupby(
    [air_quality["datetime"].dt.weekday, "location"]
)["value"].mean()

In [ ]:
# 0: 월요일, 6: 일요일
air_quality["datetime"].dt.weekday

In [ ]:
# 각 주별, 관측소별 평균 측정값 구하기
air_quality.groupby(
    [air_quality["datetime"].dt.week, "location"]
)["value"].mean()

In [ ]:
# 각 월별, 관측소별 측정 횟수 구하기
air_quality.groupby(
    [air_quality["datetime"].dt.month, "location"]
)["value"].count()

In [ ]:
# 시간별 평균 측정값 시각화
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 4))

air_quality.groupby(
    air_quality["datetime"].dt.hour
)["value"].mean().plot.bar(
    rot=0, ax=ax
)

plt.show()